In [1]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('resources/train.csv')

In [ ]:
data.isna().sum().sort_values()

In [172]:
data.drop(columns = ['OrganizationIdentifier','SmartScreen','Census_IsWIMBootEnabled','Census_ThresholdOptIn','Census_InternalBatteryType',
          'Census_IsFlightingInternal','DefaultBrowsersIdentifier','Census_ProcessorClass','PuaMode'], inplace = True)

data['AvSigVersion'] = data['AvSigVersion'].apply(lambda x:  x.replace('.', '')[0:4])

In [173]:
data = data.dropna()

In [174]:
# Labels are the values we want to predict
labels = np.array(data['HasDetections'])
# Remove the labels from the features
# axis 1 refers to the columns
features = data.drop(columns = ['HasDetections','MachineIdentifier',
       'CityIdentifier', 'Census_OEMModelIdentifier',
       'Census_FirmwareVersionIdentifier'], axis = 1)

features = pd.get_dummies(features[:100000], columns = list(features.columns).remove('Census_SystemVolumeTotalCapacity'))

In [175]:
labels = labels[:100000]

In [185]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 1)

In [186]:
## Import the random forest model.

from sklearn.ensemble import RandomForestClassifier 
## This line instantiates the model. 
rf = RandomForestClassifier() 
## Fit the model on your training data.
rf.fit(train_features, train_labels)

C:\Users\PhoenixJauregui\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [187]:
#Test Score
rf.score(test_features, test_labels)

0.57256

In [188]:
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = train_features.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

In [189]:
top_features = list(feature_importances[feature_importances['importance'] != 0].index)

In [190]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

classifier.fit(train_features[top_features], train_labels)

C:\Users\PhoenixJauregui\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [191]:
classifier.score(test_features[top_features],test_labels)

0.50764

In [184]:
unique2 = data[0:100000].nunique()